In [1]:
import fashion_recommender_color_functions_v3 as fc
import pickle 
import string
import ast
import re

import numpy as np
import pandas as pd 
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [2]:
CLEAN_PATH = "./data/data_parsed_new.csv"
TFIDF_ENCODING_PATH = "./model/data_tfidf_encodings.pkl"
TFIDF_MODEL_PATH = "./model/data_tfidf.pkl"

# Load Our Dataset
def loadData(fileName): 
    df = pd.read_csv(fileName)
    return df

def recommendationEngine(color_choice, fabric_choice, pattern_choice, N=2):
    
    #if not check_if_model_exists():
    #    print("no model..creating model file")
    #    model_maker()
      
    #color_name = fc.getColorName(color_choice)
    color_name = color_choice
    #print("selected color",color_choice, color_name)
    #search_term = color_name + " " + fabric_choice
    search_term = color_name
    #print("search 1:", search_term)
    df1 = getRecommendations(search_term, N)
    df1['color'] = color_name

    
    hexCode = fc.getHexCode(color_choice)
    print("hex code:",color_choice, hexCode)
    comp = fc.complimentary(hexCode)
    color_name = fc.getColorName(fc.rgb2hex(fc.convert(comp)))
    print("complimentary color",fc.rgb2hex(fc.convert(comp)), color_name)
    search_term = color_name #+ " " + fabric_choice
    #print("search 2:", search_term)
    df2 = getRecommendations(search_term, N)
    df2['color'] = color_name
    
    frames = [df1, df2]
    result = pd.concat(frames)
    #result = df1
    
    # what if I have duplicates.. delete them
    # sorting by first name
    result.sort_values("id", inplace = True)
 
    # dropping ALL duplicate values
    result.drop_duplicates(subset ="id",
                     keep = False, inplace = True)
    return result


def word_parser(ingreds):
    

    # The ingredient list is now a string so we need to turn it back into a list. We use ast.literal_eval
    if isinstance(ingreds, list):
        ingredients = ingreds
    else:
        ingredients = ast.literal_eval(ingreds)
        
    ingred_list = []
    for i in ingredients:
        # We split up with hyphens as well as spaces
        items = re.split(' |-', i)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        if items:
            ingred_list.append(' '.join(items)) 
    ingred_list = " ".join(ingred_list)
    return ingred_list


def getRecommendations(ingredients, N):
    """
    The recommendation system takes in a list of ingredients and returns a list of top N 
    matches based of of cosine similarity. 
    :param ingredients: a list of search terms
    :param N: the number of reccomendations returned 
    :return: top N reccomendations 
    """
    # load in tdidf model and encodings 
    with open(TFIDF_ENCODING_PATH, 'rb') as f:
        tfidf_encodings = pickle.load(f)

    with open(TFIDF_MODEL_PATH, "rb") as f:
        tfidf = pickle.load(f)

    # parse the ingredients using the ingredient_parser 
    try: 
        ingredients_parsed = word_parser(ingredients)
    except:
        ingredients_parsed = word_parser([ingredients])
    
    # use our pretrained tfidf model to encode our input ingredients
    ingredients_tfidf = tfidf.transform([ingredients_parsed])

    # calculate cosine similarity between actual recipe ingreds and test ingreds
    cos_sim = map(lambda x: cosine_similarity(ingredients_tfidf, x), tfidf_encodings)
    scores = list(cos_sim)

    # Filter top N recommendations 
    filtered_recommendations = filterRecommendations(N, scores)
    return filtered_recommendations


# Top-N recomendations order by score
def filterRecommendations(N, scores):
    # load in recipe dataset 
    df_rec = loadData(CLEAN_PATH)
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations 
    # added "dtype=" to fix a pandas dataframe error
    #recommendation = pd.DataFrame(columns = ['recipe', 'desc', 'ingredients', 'score', 'url'], dtype=object)
    #COLUMN_NAME = ['title', 'desc', 'keywords', 'img_url','score','id']
    recommendation = pd.DataFrame(columns = ['id','title', 'desc', 'score', 'keywords','img_url'], dtype=object)
    #print (recommendation)
    count = 0
    for i in top:
        #recommendation.at[count, 'url'] = df_rec['recipe_urls'][i]
        #recommendation.at[count, COLUMN_NAME[3]] = getRandomImage(df_rec[COLUMN_NAME[3]][i])
        recommendation.at[count, 'id'] = df_rec['id'][i]
        recommendation.at[count, 'img_url'] = df_rec['img_url'][i]
        recommendation.at[count, 'title'] = title_parser(df_rec['title'][i])
        recommendation.at[count, 'desc'] = title_parser(df_rec['desc'][i])
        recommendation.at[count, 'keywords'] = df_rec['keywords'][i]
        recommendation.at[count, 'score'] = "{:.3f}".format(float(scores[i])) #error here?
        count += 1
    return recommendation

In [3]:
def title_parser(title):
    #title = unidecode.unidecode(title)
    return title

def goRecommend(color_choice, fabric_choice, pattern_choice):
    results = recommendationEngine(color_choice, fabric_choice, pattern_choice)
    return results

In [6]:
#color_choice = "#c71868"
color_name = "blue"
fabric_choice = "silk"
pattern_choice = "plain"

results = goRecommend(color_name, fabric_choice, pattern_choice)
results

hex code: blue #0000ff
complimentary color #ffff00 yellow


,id,title,desc,score,keywords,img_url,color
0,43,Georgette Luckhnowi Embroidery Fabric,Aqua Blue georgette heavy embroidery in lighte...,0.587,aqua blue georgette heavy embroidery lighter blue,./images/askdevi_filename_2.jpg,blue
1,51,Georgette Luckhnowi Embroidery Fabric,Light Aqua Blue georgette heavy embroidery in ...,0.535,light aqua blue georgette heavy embroidery lig...,./images/askdevi_filename_4.jpg,blue
0,140,Iktara Zari Brocade Fabric,Mustard Yellow with zari and maroon brocade on...,0.397,mustard yellow zari maroon brocade silk,./images/askdevi_filename_22.jpg,yellow
1,226,Brocade Fabric,Old gold yellow with gold flower and stem moti...,0.370,old gold yellow gold flower stem motif brocade...,./images/askdevi_filename_51.jpg,yellow


In [ ]:


from IPython.display import display

for i in results:
    display(i)
    
from matplotlib.pyplot import imshow
%matplotlib inline

w, h = 20, 20
image = X[0].reshape(w,h).T #assuming X[0] is of shape (400,)

imshow(image, cmap='gray')

from IPython.display import display
display({'text/html': '<h1>Hello World</h1>', 'text/plain': 'Hello World'}, raw=True)
